In [1]:
# pandas
import pandas as pd
from pandas import Series, DataFrame

In [2]:
# numpy, matplotlib, seaborn
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline

In [3]:
# machine learning
from sklearn.linear_model import LogisticRegression

In [4]:
train_df = pd.read_csv("./train.csv")
test_df = pd.read_csv("./test.csv")

In [5]:
# what does the data look like
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
test_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [7]:
# Variable     Definition            Key
# survival     Survival              0 = No, 1 = Yes
# pclass       Ticket class          1 = 1st, 2 = 2nd, 3 = 3rd
# sex          Sex
# Age          Age in years
# sibsp        # of siblings / spouses aboard the Titanic
# parch        # of parents / children aboard the Titanic
# ticket       Ticket number
# fare         Passenger fare
# cabin        Cabin number
# embarked     Port of Embarkation   C = Cherbourg, Q = Queenstown, S = Southampton

In [8]:
# 移除那些我认为和预测没有关系的column
# embarked 从哪里登船我认为和预测是否生是无关的
train_df = train_df.drop(['Embarked'], axis=1)
test_df = test_df.drop(['Embarked'], axis=1)
# ticket number 票根的序列号
train_df = train_df.drop(['Ticket'], axis=1)
test_df = test_df.drop(['Ticket'], axis=1)

In [29]:
train_df = train_df.drop(['Name'], axis=1)
test_df = test_df.drop(['Name'], axis=1)

In [9]:
# 查看每个列中是否存在空值
train_df.isnull().any()

PassengerId    False
Survived       False
Pclass         False
Name           False
Sex            False
Age             True
SibSp          False
Parch          False
Fare           False
Cabin           True
dtype: bool

In [10]:
test_df.isnull().any()

PassengerId    False
Pclass         False
Name           False
Sex            False
Age             True
SibSp          False
Parch          False
Fare            True
Cabin           True
dtype: bool

# 对空值的处理

In [11]:
# age
# 如果存在空值，查看空值行占该列的比例
# age for training DataFrame
train_df[train_df['Age'].isnull().values == True].shape[0] / train_df.shape[0]

0.19865319865319866

In [12]:
# age fro test DataFrame
test_df[test_df['Age'].isnull().values == True].shape[0] / test_df.shape[0]

0.20574162679425836

In [13]:
# 由于age的空值在train_df和test_df中的比例都不是很高，所以我决定以随机值来填充age，使这一列没有空值
average_age_train = train_df["Age"].mean()
std_age_train = train_df["Age"].std()
count_nan_age_train = train_df["Age"].isnull().sum()
average_age_test    = test_df["Age"].mean()
std_age_test        = test_df["Age"].std()
count_nan_age_test  = test_df["Age"].isnull().sum()
rand_train = np.random.randint(average_age_train - std_age_train, average_age_train + std_age_train, 
                              size=count_nan_age_train)
rand_test = np.random.randint(average_age_test - std_age_test, average_age_test + std_age_test, 
                             size=count_nan_age_test)

In [14]:
train_df["Age"][np.isnan(train_df["Age"])] = rand_train
test_df["Age"][np.isnan(test_df["Age"])] = rand_test

/home/esta/anaconda2/envs/py3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/esta/anaconda2/envs/py3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [15]:
# 这里比较填充之前和填充之后的年龄分布的过程被我省略，正常情况下应该去比较，如果分布类似，证明填充是有效的

In [17]:
# Cabin
print(train_df[train_df['Cabin'].isnull().values == True].shape[0] / train_df.shape[0])
print(test_df[test_df['Cabin'].isnull().values == True].shape[0] / test_df.shape[0])

0.7710437710437711
0.7822966507177034


In [18]:
# 我们看到Cabin这一行有太多的空值，所以我将其直接drop
train_df = train_df.drop(['Cabin'], axis=1)
test_df = test_df.drop(['Cabin'], axis=1)

In [19]:
# Fare
# 在Fare中，test_df的行有缺失，使用中位数来填充
test_df["Fare"].fillna(test_df["Fare"].median(), inplace=True)

In [20]:
train_df.isnull().any()

PassengerId    False
Survived       False
Pclass         False
Name           False
Sex            False
Age            False
SibSp          False
Parch          False
Fare           False
dtype: bool

In [22]:
test_df.isnull().any()

PassengerId    False
Pclass         False
Name           False
Sex            False
Age            False
SibSp          False
Parch          False
Fare           False
dtype: bool

In [23]:
# 空值填充完毕

# 转换列的数据类型

In [24]:
# 需要将列的数据类型转化为估计器可以识别的类型

In [25]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            891 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Fare           891 non-null float64
dtypes: float64(2), int64(5), object(2)
memory usage: 62.7+ KB


In [26]:
# 为了提高模型的训练速度，将float64转为int64
train_df['Fare']   = train_df['Fare'].astype(int)
test_df['Fare']    = test_df['Fare'].astype(int)
train_df['Age']    = train_df['Age'].astype(int)
test_df['Age']     = test_df['Age'].astype(int)

In [30]:
# Sex
def trans_sex(sex):
    if sex == "male":
        return 0
    else:
        return 1
train_df['Sex'] = train_df['Sex'].apply(trans_sex)
test_df['Sex'] = test_df['Sex'].apply(trans_sex)

In [31]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Sex            891 non-null int64
Age            891 non-null int64
SibSp          891 non-null int64
Parch          891 non-null int64
Fare           891 non-null int64
dtypes: int64(8)
memory usage: 55.8 KB


In [32]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 7 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Sex            418 non-null int64
Age            418 non-null int64
SibSp          418 non-null int64
Parch          418 non-null int64
Fare           418 non-null int64
dtypes: int64(7)
memory usage: 22.9 KB


In [36]:
X_train = train_df.drop(['Survived', 'PassengerId'],axis=1)
Y_train = train_df['Survived']
X_test  = test_df.drop('PassengerId',axis=1).copy()

In [39]:
# Logistic Regression

logreg = LogisticRegression()

logreg.fit(X_train, Y_train)

Y_pred = logreg.predict(X_test)

In [40]:
test_df["Survived"] = Series(Y_pred)

In [41]:
test_df.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Survived
0,892,3,0,34,0,0,7,0
1,893,3,1,47,1,0,7,0
2,894,2,0,62,0,0,9,0
3,895,3,0,27,0,0,8,0
4,896,3,1,22,1,1,12,1


In [42]:
result_df = test_df.drop(["Pclass", "Age", "SibSp", "Parch", "Fare", "Sex"], axis=1)

In [43]:
result_df.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1


In [44]:
result_df.to_csv("./result.csv", index=False)